In [1]:
import cv2
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dropout, BatchNormalization
from keras import models
from keras.layers import Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Flatten

from tensorflow.python.keras.preprocessing.sequence import pad_sequences 
from tensorflow.python.keras.preprocessing.text import Tokenizer

from keras.layers import Dense, Embedding, LSTM, Dropout, Conv1D, MaxPooling1D

import re

import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

Using TensorFlow backend.


In [2]:
df=pd.read_csv('Reviews.csv')
print(df.head())

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

In [3]:
score=df['Score']
score=np.array(score)

In [4]:
#adjust scores
for i in range(len(score)):
    if(score[i]>3):
        score[i]=1
    else:
        score[i]=0

In [5]:
#Create a dataframe to be appended
X=pd.DataFrame(score)
#Add the dataframe
df['New']=X
#drop the score column
df=df.drop(['Score'],axis=1)
df=df.rename(columns={'New' : 'Score'})

df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Time,Summary,Text,Score
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1


In [6]:
# df_new=df.sample(7000)
# X=df_new.iloc[:,0:df_new.shape[1]-1]
# y=df_new.iloc[:,df_new.shape[1]-1]

In [7]:
m0 = df['Score']==0
m1 = df['Score']==1

df_0=df.where(m0)
df_0=df_0.dropna()

df_1=df.where(m1)
df_1=df_1.dropna()

# print(df_0.shape)

In [8]:
df_0=df_0.sample(5000)
df_1=df_1.sample(5000)

In [9]:
frames = [df_0, df_1]
df_new=pd.concat(frames)

In [10]:
df_new=df_new.sample(frac=1)
# X=df_new.iloc[:,0:df_new.shape[1]-1]
# y=df_new.iloc[:,df_new.shape[1]-1]

In [11]:
# nltk libraries
stop_words = set(stopwords.words('english'))
lemmatiser = WordNetLemmatizer()

In [12]:
# returns the Part-Of-Speech(pos) tag which is required as an argument for
# lemmatizer for determiing the context of the word for lemmatization
def get_wordnet_pos(pos_tag):

    if pos_tag.startswith('J'):  
        return 'a' # adjective
    elif pos_tag.startswith('V'): 
        return 'v' # verb
    elif pos_tag.startswith('R'):
        return 'r' # adverb
    else:           
        return 'n' # fallback to noun

In [13]:
#Preprocess the entire text
def preprocess_text(text):
    
    # lowercase the entire text
    text = text.lower()
    
    # remove all html tags from dataset
    text = re.sub(r'<.*?>',"",text)
    
    # Replace special characters and punctuation by a 
    # blank ( except for full stop(.))
    text = re.sub('[^a-z0-9 \.]', '', text)
    
    # split the text into words
    words = word_tokenize(text) 
    
    # remove stop words like to, an, the, of, over, his, her, there, their ...
    words = [w for w in words if w =="not" or not w in stop_words ]
    
    # get pos tagging for words, required by lemmatizer
    words_pos = pos_tag(words) 
    
    lemmatized_words = []
    
    for word, tag in words_pos:
        pos = get_wordnet_pos(tag)
        lemmatized_word = lemmatiser.lemmatize(word, pos=pos)
        lemmatized_words.append(lemmatized_word)
    
    text = " ".join(lemmatized_words)
    
    return text

In [14]:
df_summary=df_new['Summary']
df_text=df_new['Text']

df_text=np.array(df_text)
df_summary=np.array(df_summary)

In [15]:
n=len(df_new)
reviews=[]
for i in range(n):
    
    text = str(df_text[i])
    title = str(df_summary[i])
    
    review = title + " " + text
    review = preprocess_text(review)
    reviews.append(review)

In [16]:
df_new2=df_new.drop(['Summary','Text'],axis=1)
df_new2

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Time,Score
507211,507212.0,B00127T7TK,A2BQ9N8O1QA00N,C. D. Rickert,2.0,3.0,1.294618e+09,0.0
495822,495823.0,B000PJAQR8,A27DUQTEV26CXG,saywut,1.0,1.0,1.279152e+09,1.0
242795,242796.0,B000BF3AII,A24R1AOSCZ4CO7,steveeb,0.0,0.0,1.275178e+09,1.0
120352,120353.0,B005K4Q37A,AWEBTQUXK7YB4,Steven Pretzman,2.0,3.0,1.344211e+09,0.0
128467,128468.0,B007L3NVKU,A3S84XXELJ0YX2,R. A. Balkam,0.0,0.0,1.322698e+09,1.0
179099,179100.0,B000LKV2KQ,AQ31FHYKXQ8C5,ebookluvr,1.0,1.0,1.336608e+09,1.0
174216,174217.0,B000LL0RKG,A2YKK9K1ARLSYO,Kristine Prizzio,0.0,0.0,1.246234e+09,1.0
178043,178044.0,B000UXDHGQ,A1Z7Y2GMAP9SRY,"M. Thompson ""Dyson Diva""",3.0,3.0,1.258243e+09,1.0
210755,210756.0,B001EQ4OWO,A2DOQ9ERI5ELAZ,Jessica,0.0,0.0,1.339718e+09,1.0
138918,138919.0,B0041CIR62,A319Y83RT0MRVR,AIROLF,0.0,1.0,1.329350e+09,1.0


In [17]:
df_new2['Review']=pd.DataFrame(reviews).values
df_new2

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Time,Score,Review
507211,507212.0,B00127T7TK,A2BQ9N8O1QA00N,C. D. Rickert,2.0,3.0,1.294618e+09,0.0,ok syrup thick mean mix shave ice rather pour ...
495822,495823.0,B000PJAQR8,A27DUQTEV26CXG,saywut,1.0,1.0,1.279152e+09,1.0,addictive apparently addictive property ive me...
242795,242796.0,B000BF3AII,A24R1AOSCZ4CO7,steveeb,0.0,0.0,1.275178e+09,1.0,try tortilla soup rice . family love dice chik...
120352,120353.0,B005K4Q37A,AWEBTQUXK7YB4,Steven Pretzman,2.0,3.0,1.344211e+09,0.0,sweet poor taste . sweet make sick . stuff awf...
128467,128468.0,B007L3NVKU,A3S84XXELJ0YX2,R. A. Balkam,0.0,0.0,1.322698e+09,1.0,delish love coffee . like strong coffee right ...
179099,179100.0,B000LKV2KQ,AQ31FHYKXQ8C5,ebookluvr,1.0,1.0,1.336608e+09,1.0,good good love stuff popcorn contain lots grea...
174216,174217.0,B000LL0RKG,A2YKK9K1ARLSYO,Kristine Prizzio,0.0,0.0,1.246234e+09,1.0,great vacation not wealthy person mean . howev...
178043,178044.0,B000UXDHGQ,A1Z7Y2GMAP9SRY,"M. Thompson ""Dyson Diva""",3.0,3.0,1.258243e+09,1.0,yummy great tasty fresh grapefruit difficult f...
210755,210756.0,B001EQ4OWO,A2DOQ9ERI5ELAZ,Jessica,0.0,0.0,1.339718e+09,1.0,bag amaze love good environment . recently mov...
138918,138919.0,B0041CIR62,A319Y83RT0MRVR,AIROLF,0.0,1.0,1.329350e+09,1.0,good gluten free alternative follow direction ...


In [18]:
df_new2=df_new2.iloc[:,0:]
df_new2.to_csv(r'amazon_reviews_cleaned.csv')